In [1]:
import numpy as np
from numpy.array_api import linspace
# import mne
from scipy.io import loadmat
from scipy.signal import decimate, butter, filtfilt
import pandas as pd
import matplotlib.pyplot as plt
import importlib

from EEG_TDA_Pipeline import highcut, target_fs
from eeg_utils import *
%matplotlib inline

/tmp/ipykernel_27993/671407191.py:2: UserWarning: The numpy.array_api submodule is still experimental. See NEP 47.
  from numpy.array_api import linspace


## Loading the data

In [4]:
# Data name
file_name = str("sub_i4oK0F_HoldL_MedOff_run1_LFP_Hilbert.mat")

# Subject code name
name = "i4oK0F"

In [15]:
# Imports the .mat file as a DataFrame
df, left_lfp, right_lfp, left_name, right_name = mat_to_dataframe("sub-0cGdk9_HoldL_MedOff_run1_LFP_Hilbert/" + file_name)

--- Processing file: sub_i4oK0F_HoldL_MedOff_run1_LFP_Hilbert.mat ---
Found data matrix. Key: 'selected_channels_signal', Shape: (2, 1509667)
Found labels. Key: 'target_labels'
Labels extracted: ['LFP-right-23', 'LFP-left-78']


--- DataFrame Created Successfully ---
          LFP-right-23  LFP-left-78
Time (s)                           
0.0000        1.253931    -0.608706
0.0005        0.386463    -0.819621
0.0010       -0.496690    -0.184820
0.0015        0.670664    -0.899828
0.0020        1.982702     1.812119

Successfully extracted left LFP channel: 'LFP-left-78'
Successfully extracted right LFP channel: 'LFP-right-23'


## Preprocessing

In [8]:
# Set the necessary variables

original_fs = 2000 # Hz (original sampling rate)
lowcut = 4 # Hz (cut-off for high-pass filter)
highcut = 48 # Hz (cut-off for low-pass filter)

In [9]:
# Applying band-pass filter

left_filtered = butter_bandpass_filter(data=left_lfp,
                              lowcut=lowcut,
                              highcut=highcut,
                              fs=original_fs,
                              order=5)

right_filtered = butter_bandpass_filter(data=right_lfp,
                              lowcut=lowcut,
                              highcut=highcut,
                              fs=original_fs,
                              order=5)

In [17]:
# Downsampling the signal

target_fs = 100 # Hz
ftype = "fir" # or "iir" (anti-aliasing filter)

if original_fs % target_fs != 0:
    raise ValueError("Original fs must be an integer multiple of Target fs.")

q = original_fs // target_fs

left = decimate(x=left_filtered, q=q, ftype=ftype, zero_phase=True)

# Verify the result
print(f"Original number of left samples: {len(left_filtered)}")
print(f"Downsampled number of left samples: {len(left)}")

right = decimate(x=right_filtered, q=q, ftype=ftype, zero_phase=True)

# Verify the result
print("-------")
print(f"Original number of right samples: {len(right_filtered)}")
print(f"Downsampled number of right samples: {len(right)}")
print(f"New sampling rate: {target_fs} Hz")

Original number of left samples: 1509667
Downsampled number of left samples: 75484
-------
Original number of right samples: 1509667
Downsampled number of right samples: 75484
New sampling rate: 100 Hz
